In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import os
import sys
import random as rn
rn.seed(42)

sys.path.append(os.path.abspath(os.pardir))

import numpy as np
np.random.seed(42)
from sklearn.model_selection import StratifiedShuffleSplit

from tdparse.helper import read_config, full_path
from tdparse.parsers import semeval_14
from tdparse.data_types import TargetCollection, Target
from tdparse import write_data

# Creating Training and Test sets for the YouTuBean Dataset
We show how we created the Training and Test sets for this dataset.

The original Dataset can be downloaded from [here](https://github.com/epochx/opinatt) and the accompying paper can be found [here](https://www.aclweb.org/anthology/W17-5213). As Marrese-Taylor et al. Evaluated their models on 5 fold cross validation they do not have one train, test set therefore we create a 70% train and 30% test dataset, we then save the new train and test dataset in XML format just like the original format. This format is the same as [SemEval 2014](http://alt.qcri.org/semeval2014/task4/) XML format.

First ensure the following has been done:
1. Download the dataset (called samsung_galaxy_s5.xml)
2. Ensure in the [config.yaml](../config.yaml) file that the following values have the correct file paths:
  1. youtubean = the file path to the downloaded dataset (samsung_galaxy_s5.xml)
  2. youtubean_train = the file path that you would like the new training dataset to go
  3. youtubean_test = the file path that you would like the new test dataset to go

In [2]:
# YouTube dataset
youtubean_with_conflicts = semeval_14(full_path(read_config('youtubean')), conflict=True)
youtubean = semeval_14(full_path(read_config('youtubean')))

Shown above is two ways to parse the data. The first one contains labels called conflicts which is when the sentiment is both positive and negative. The second removes the conflict labels. The original analysis by Marrese-Taylor mapped the conflict labels to neutral labels. However all of our experiments we remove the conflict labelled data.

We thought it would be best to show you here how you can parse the data differently. Also we parse the data with conflicts so that we can show that we have parsed the data correctly as the statitics from Marrese-Taylor paper assume you include the conflict data. In there paper they state that there are 525 uniqie aspect terms and that is exactly what we got as shown here:

{{youtubean_with_conflicts.number_unique_targets()}}

In the dataset that we are using which contains **no conflict labels** there are: {{len(youtubean)}} targets

In [3]:
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

youtubean_data = np.asarray(youtubean.data_dict())
youtubean_sentiment = np.asarray(youtubean.sentiment_data())
for train_indexs, test_indexs in splitter.split(youtubean_data, youtubean_sentiment):
    train_data = youtubean_data[train_indexs]
    test_data = youtubean_data[test_indexs]
    
convert_to_targets = lambda data: [Target(**target) for target in data]
youtubean_train = TargetCollection(convert_to_targets(train_data))
youtubean_test = TargetCollection(convert_to_targets(test_data))

The dataset has now been split with respect to the class labels so each class label is represented equally in the train and test splits which can be shown here:

Train Data ratio: **{{youtubean_train.ratio_targets_sentiment()}}**
Train Data raw values: **{{youtubean_train.no_targets_sentiment()}}**

Test Data ratio: **{{youtubean_test.ratio_targets_sentiment()}}**
Test Data raw values: **{{youtubean_test.no_targets_sentiment()}}**

Original Data ratio: **{{youtubean.ratio_targets_sentiment()}}**  
Original Data raw values: **{{youtubean.no_targets_sentiment()}}**

We now save the data to XML file format which is the same as the SemEval data format.

In [4]:
write_data.semeval_14(full_path(read_config('youtubean_train')), youtubean_train)
write_data.semeval_14(full_path(read_config('youtubean_test')), youtubean_test)